In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import *
import pyspark.sql.functions as F
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql.functions import to_timestamp

# from pyspark.sql.functions import col, udf
from pyspark.sql.functions import col, unix_timestamp, to_date

In [3]:
spark = pyspark.sql.SparkSession.builder.master("local").getOrCreate()
spark

21/12/07 14:27:21 WARN Utils: Your hostname, LAPTOP-T2T1G8JL resolves to a loopback address: 127.0.1.1; using 172.22.10.85 instead (on interface eth0)
21/12/07 14:27:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/07 14:27:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/12/07 14:27:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/12/07 14:27:24 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
21/12/07 14:27:24 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [4]:
# Let's define our schema
schema = StructType([\
    StructField("date", StringType(), True),\
    StructField("time", StringType(), True),\
    StructField("company", StringType(), True),\
    StructField("level", StringType(), True),\
    StructField("title", StringType(), True),\
    StructField("totalyearlycompensation", IntegerType(), False),\
    StructField("location", StringType(), True),\
    StructField("yearsofexperience", FloatType(), False),\
    StructField("yearsatcompany", FloatType(), False),\
    StructField("tag", StringType(), True),\
    StructField("basesalary", IntegerType(), False),\
    StructField("stockgrantvalue", IntegerType(), False),\
    StructField("bonus", IntegerType(), False),\
    StructField("gender", StringType(), True),\
    StructField("cityid", StringType(), True),\
    StructField("dmaid", StringType(), True),\
    StructField("race", StringType(), True),\
    StructField("education", StringType(), True)])

# Load and parse the data file, converting it to a DataFrame.
data = spark.read.format("csv")\
    .option("header", "false")\
    .option("delimiter", "\t")\
    .schema(schema)\
    .load("data/seperated_time_data/cleaned.txt")
data.show(n=5)

+----------+--------+---------+-----+--------------------+-----------------------+-----------------+-----------------+--------------+---+----------+---------------+-----+------+------+-----+----+---------+
|      date|    time|  company|level|               title|totalyearlycompensation|         location|yearsofexperience|yearsatcompany|tag|basesalary|stockgrantvalue|bonus|gender|cityid|dmaid|race|education|
+----------+--------+---------+-----+--------------------+-----------------------+-----------------+-----------------+--------------+---+----------+---------------+-----+------+------+-----+----+---------+
|2017-06-07|   11:33|   ORACLE|   L3|     Product Manager|                 127000| Redwood City, CA|              1.5|           1.5| NA|    107000|          20000|10000|    NA|  7392|  807|  NA|       NA|
|2017-06-10|   17:11|     EBAY| SE 2|   Software Engineer|                 100000|San Francisco, CA|              5.0|           3.0| NA|    141907|              0|    0|    NA

In [5]:
# nice cheat sheet https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf

# I think this link is a more explained version of the above link https://towardsdatascience.com/pyspark-and-sparksql-basics-6cb4bf967e53


In [6]:
# getting a dataframe for how many times each job title occurs
all_titles = data.groupby("title").count()

# sorting the dataframe and taking the top 10
# this link helped : https://sparkbyexamples.com/pyspark/pyspark-orderby-and-sort-explained/
all_titles.sort(col("count").desc())
most_common_titles = all_titles.sort(col("count").desc()).limit(10)

# trying to take all job titles as a list
most_common_titles_list = most_common_titles.rdd.map(lambda x: x[0]).collect()

In [7]:
# filter data by title in that list
filtered_data = data.where(F.col("title").isin(most_common_titles_list))

# groupby title and find average
avg_salary_per_job = filtered_data.groupby("title").mean("basesalary")

# round decimals to two places
rounded_avg_salary_per_job = avg_salary_per_job.withColumn("avg(basesalary)", F.round(avg_salary_per_job["avg(basesalary)"], 2))

# sort the dataframe
rounded_avg_salary_per_job.sort(col("avg(basesalary)").desc()).show()

+--------------------+---------------+
|               title|avg(basesalary)|
+--------------------+---------------+
|Software Engineer...|       183349.5|
|     Product Manager|      156802.03|
|Technical Program...|      156660.26|
|  Solution Architect|      147041.96|
|      Data Scientist|      143449.14|
|   Hardware Engineer|      142595.33|
|    Product Designer|      142218.32|
|   Software Engineer|      137701.16|
|Management Consul...|       136195.7|
|    Business Analyst|      103986.13|
+--------------------+---------------+

